In [ ]:
#IMPORTS FOR SEARCH ENGINE
import pandas as pd
from time import sleep
from tqdm import tqdm
import xml.etree.ElementTree as ET
!pip install keybert
from keybert import KeyBERT
import random
import time
import warnings
from IPython.display import display, HTML
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.lib.display import html_escape

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 67.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 20.0 MB/s eta 0:00:00
  Created wheel for keybert: filename=keybert-0.7.0-py3-none-any.whl size=23794 sha256=8eff3854689ec3f867d3e528758ee5ae1f73f980613fa115be3cedf50e19883f
  Stored in directory: /root/.cache/pip/wheels/68/aa/41/82025d89b0eb97484c9ac7d527abf596765c41733af79f86b0
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=be989cb36c30e75c307fde9633dabb1abfb66aa5ea72a9

In [ ]:
pd.options.mode.chained_assignment = None

In [ ]:
### Reading the topic file from GDrive  ###

In [ ]:
topic_files = pd.read_xml("topics2021.xml")

In [ ]:
### SETTING UP TRIALS CSV - IMPORT LOCALLY ###

In [ ]:
trials = pd.read_csv('clin_trials_14973.csv', header= 0, index_col= False)
trials = trials.iloc[:, 1:]

In [ ]:
#### PREPROCESSING MINAGE AND MAXAGE COLUMNS SO ALL ARE INTEGERS IN YEARS

In [ ]:
trials['minage'] = trials['minage'].fillna(0)
trials['maxage'] = trials['maxage'].fillna(200)

In [ ]:
#### FOR MONTHS ####
age_mon_min = trials[trials['minage'].str.contains('Month', na=False)]
age_mon_min ['minage'] = age_mon_min['minage'].str.replace('\D+', '').astype(int).div(12).round(2)
trials.update(age_mon_min, join='left', overwrite=True, filter_func=None, errors='ignore')

age_mon_max = trials[trials['maxage'].str.contains('Month', na=False)]
age_mon_max ['maxage'] = age_mon_max['maxage'].str.replace('\D+', '').astype(int).div(12).round(2)
trials.update(age_mon_max, join='left', overwrite=True, filter_func=None, errors='ignore')


#### FOR WEEKS ####
age_week_min = trials[trials['minage'].str.contains('Week', na=False)]
age_week_min ['minage'] = age_week_min['minage'].str.replace('\D+', '').astype(int).div(52).round(2)
trials.update(age_week_min, join='left', overwrite=True, filter_func=None, errors='ignore')

age_week_max = trials[trials['maxage'].str.contains('Week', na=False)]
age_week_max ['maxage'] = age_week_max['maxage'].str.replace('\D+', '').astype(int).div(52).round(2)
trials.update(age_week_max, join='left', overwrite=True, filter_func=None, errors='ignore')


#### FOR YEARS ####
min_years = trials[trials['minage'].str.contains('Year', na=False)]
min_years['minage'] = min_years['minage'].str.replace('\D+', '').astype(int)
trials.update(min_years, join='left', overwrite=True, filter_func=None, errors='ignore')

max_years = trials[trials['maxage'].str.contains('Year', na=False)]
max_years['maxage'] = max_years['maxage'].str.replace('\D+', '').astype(int)
trials.update(max_years, join='left', overwrite=True, filter_func=None, errors='ignore')


#### FOR DAYS ####
age_days_min = trials[trials['minage'].str.contains('Day', na=False)]
age_days_min['minage'] = age_days_min['minage'].str.replace('\D+', '').astype(int).div(365).round(2)
trials.update(age_days_min, join='left', overwrite=True, filter_func=None, errors='ignore')

age_days_max = trials[trials['maxage'].str.contains('Day', na=False)]
age_days_max['maxage'] = age_days_max['maxage'].str.replace('\D+', '').astype(int).div(365).round(2)
trials.update(age_days_max, join='left', overwrite=True, filter_func=None, errors='ignore')

#### FOR HOURS ####
age_hour_max = trials[trials['maxage'].str.contains('Hour', na=False)]
age_hour_max['maxage'] = age_hour_max['maxage'].str.replace('\D+', '').astype(int).div(8760).round(2)
trials.update(age_hour_max, join='left', overwrite=True, filter_func=None, errors='ignore')

age_min_max = trials[trials['maxage'].str.contains('Minute', na=False)]
age_min_max['maxage'] = age_min_max['maxage'].str.replace('\D+', '').astype(int).div(525600).round(2)
trials.update(age_min_max, join='left', overwrite=True, filter_func=None, errors='ignore')

In [ ]:
trials = trials.drop_duplicates()

In [ ]:
#### FINAL DATAFRAME OF DOCUMENT COLLECTION READY FOR PARSING ####

In [ ]:
trials

,title,id,brief,gender,healthy,minage,maxage,condition,intervention,inclusion,exclusion
0,Efficacy of Tachosil as Dural Sealant Compared...,NCT00999999,Main objective of this trial is to ass...,All,No,18.0,200,NaN,Procedure Craniotomy supra- or infratentorial...,- Neurosurgical indication for craniotomy and...,Criteria: - Presence of subdural...
1,Low Dose Cyclophosphamide Treats Genital Warts,NCT00999986,Condylomata acuminata (CA) caused by h...,All,Accepts Healthy Volunteers,18.0,60.0,Warts Condylomata Acuminata,Drug cyclophosphamide 50 mg oral per day for ...,- Clinical diagnosis of large genital warts,Criteria: - HPV type 6 or 11 was...
2,Study on the Effect of Mitomycin C 0.02% on th...,NCT00999973,In this study 126 eyes in 63 patients ...,All,NaN,19.0,60.0,Myopia,Drug Mitomycin C 0.02% Mitomycin c 0.02%,- 19 < age < 60 - Refractory er...,criteria: - Keratoconus ...
3,Study to Evaluate the Clinical Impact of a Lap...,NCT00999960,To evaluate the clinical impact of an ...,All,No,0,200,Prostatic Neoplasms Neoplasms,Procedure Laparoscopic Radical Prostatectomy ...,- Surgeon beginning his clinical experience i...,Criteria: - Surgeon who does not...
4,Arrhythmogenic Right Ventricular Dysplasia/Car...,NCT00999947,The purpose of this trial is to study ...,All,No,10.0,200,Cardiomyopathies Arrhythmogenic Right Ventri...,NaN,- Patient with DVDA diagnostic confirmed ...,Criteria: - Impossible to unders...
...,...,...,...,...,...,...,...,...,...,...,...
16865,Testosterone Gel in Preventing Weakness Caused...,NCT00631137,RATIONALE: Testosterone gel may be eff...,Male,No,18.0,120.0,Nervous System Neoplasms Central Nervous Sys...,Drug testosterone gel applied to skin Applica...,NaN,NaN
16866,Positron Emission Tomography-Computed Tomograp...,NCT00643591,The objectives of the trial are: ...,All,No,19.0,200,Glioma Astrocytoma,Other imaging (dynamic PET-CT scan) dynamic P...,- Histologically confirmed gliomas III - IV (...,Criteria: - Not histologically c...
16867,A Study of XL765 (SAR245409) in Combination Wi...,NCT00704080,The purpose of this study is to determ...,All,No,18.0,200,Glioblastoma Glioma,Drug XL765 (SAR245409) Gelatin capsules suppl...,- Histologically confirmed intracranial Grade...,Criteria: - Progressed while on ...
16868,Temozolomide All-Case-Registered Surveillance ...,NCT00705198,The purpose of this study is to confir...,All,No,0,200,Glioma Astrocytoma,Drug Temozolomide (for newly diagnosed malign...,- All patients treated with temozolomide for ...,Criteria: - Patients with a hist...


In [ ]:
### PARSING THE RELEVANCE CSV INTO DFS GIVEN GROUND TRUTH RELEVANCE OF 2###

In [ ]:
def rel_df(qrelsdoc, relno):
  rel = None
  rel = pd.read_csv(qrelsdoc, header= None)
  relev = [i for i in rel[0]]
  columns = ["single"]

  final = None
  final = pd.DataFrame(relev, columns= columns)
  final[['Topic_no', '0', 'NCT_no', 'Relevance']] = final['single'].str.split(' ', n=3, expand=True)
  final = final.drop(columns=['single', '0'])
  relevant_rows = final[final['Relevance'] == str(relno)]
  non_relevant_rows = final[final['Relevance'] != str(relno)]


  return final, relevant_rows, non_relevant_rows

In [ ]:
reldf, relevant, non_relevant = rel_df('qrels2021.csv', 2)

In [ ]:
#### CHOOSING THE FILES FROM 16K SAMPLE ONLY IF THEY EXIST IN THE GROUND TRUTH ####
#### (This is so that every document which is indexed has an attached ground truth score for at least one topic) ####

In [ ]:
trials = trials[trials['id'].isin(reldf['NCT_no'])]

In [ ]:
#### PARSING THESE FILES INTO LISTS OF TUPLES FOR ES ENGINE #####

In [ ]:
def dataparser(trials, topic_files):
  title = [i for i in trials['title'].astype('str')]
  id = [i for i in trials['id'].astype('str')]
  brief = [i for i in trials['brief'].astype('str')]
  gender = [i for i in trials['gender'].astype('str')]
  healthy = [i for i in trials['healthy'].astype('str')]
  minage = [i for i in trials['minage'].astype('int')]
  maxage = [i for i in trials['maxage'].astype('int')]
  condition = [i for i in trials['condition'].astype('str')]
  intervention = [i for i in trials['intervention'].astype('str')]
  inclusion = [i for i in trials['inclusion'].astype('str')]
  exclusion = [i for i in trials['exclusion'].astype('str')]

  topic_list = []
  topic_list = [i for i in topic_files["topic"]]

  return [(title, id, brief, gender, healthy, minage, maxage, condition, intervention, inclusion, exclusion)
            for title, id, brief, gender, healthy, minage, maxage, condition, intervention, inclusion, exclusion
            in  zip(title, id, brief, gender, healthy, minage, maxage, condition, intervention, inclusion, exclusion)], topic_list


In [ ]:
trials_parsed, topics = dataparser(trials, topic_files)

In [ ]:
### SETTING UP ES ENVIRONMENT ###

In [ ]:
!wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz
!wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz.sha512
!tar -xzf elasticsearch-oss-7.9.2-linux-x86_64.tar.gz
!sudo chown -R daemon:daemon elasticsearch-7.9.2/
!shasum -a 512 -c elasticsearch-oss-7.9.2-linux-x86_64.tar.gz.sha512

elasticsearch-oss-7.9.2-linux-x86_64.tar.gz: OK


In [ ]:
!pip install elasticsearch==7.9.1 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.2/219.2 kB 14.6 MB/s eta 0:00:00


In [ ]:
!pip freeze | grep elasticsearch

elasticsearch==7.9.1


In [ ]:
%%bash --bg
sudo -H -u daemon elasticsearch-7.9.2/bin/elasticsearch

In [ ]:
%%bash
ps -ef | grep elasticsearch

root        1138    1136  0 21:03 ?        00:00:00 sudo -H -u daemon elasticsearch-7.9.2/bin/elasticsearch
daemon      1139    1138  0 21:03 ?        00:00:00 /bin/bash elasticsearch-7.9.2/bin/elasticsearch
daemon      1158    1139  0 21:03 ?        00:00:00 /bin/bash elasticsearch-7.9.2/bin/elasticsearch
daemon      1159    1158  0 21:03 ?        00:00:00 /content/elasticsearch-7.9.2/jdk/bin/java -version
root        1167    1148  0 21:03 ?        00:00:00 grep elasticsearch


In [ ]:
### START ES SERVER ###

In [ ]:
time.sleep(20)
!curl -X GET "http://localhost:9200"

{
  "name" : "c30c5fc7bcde",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "wNZ4c2oaQGCZgewZxVvmSg",
  "version" : {
    "number" : "7.9.2",
    "build_flavor" : "oss",
    "build_type" : "tar",
    "build_hash" : "d34da0ea4a966c4e49417f2da2f244e3e97b4e6e",
    "build_date" : "2020-09-23T00:45:33.626720Z",
    "build_snapshot" : false,
    "lucene_version" : "8.6.2",
    "minimum_wire_compatibility_version" : "6.8.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


In [ ]:
from elasticsearch import Elasticsearch
es = Elasticsearch("http://localhost:9200")

if es.ping():
  print('ES instance working')
else:
  print('ES instance not working')

ES instance working


In [ ]:
### MAPPING ES ###

In [ ]:
#### DELETING ANY PREVIOUSLY CREATED INDEX TO AVOID STACKING ####

In [ ]:
!curl -X DELETE "localhost:9200/clinical_trials"

{"error":{"root_cause":[{"type":"index_not_found_exception","reason":"no such index [clinical_trials]","resource.type":"index_or_alias","resource.id":"clinical_trials","index_uuid":"_na_","index":"clinical_trials"}],"type":"index_not_found_exception","reason":"no such index [clinical_trials]","resource.type":"index_or_alias","resource.id":"clinical_trials","index_uuid":"_na_","index":"clinical_trials"},"status":404}

In [ ]:
#### CREATING THE INDEX "clinical_trials" WITH ALL DOCUMENT FIELD MAPPINGS ####

In [ ]:
request_body = {
    'settings': {
        'number_of_shards': 1,
        'number_of_replicas': 1,

    },
    'mappings': {
          'properties': {
              'title': {'type': 'text'},
              'id' : {'type': 'text'},
              'brief' : {'type': 'text'},
              'gender' : {'type': 'text'},
              'healthy' : {'type': 'text'},
              'minage' : {'type': 'float'},
              'maxage' : {'type': 'float'},
              'condition' : {'type': 'text'},
              'intervention' : {'type': 'text'},
              'inclusion' : {'type': 'text'},
              'exclusion' : {'type' : 'text'},
          }
    }
}

index_name = 'clinical_trials'
try:
  es.indices.get(index_name)
  print('index {} already exists'.format(index_name))
except:
  print('creating index {}'.format(index_name))
  es.indices.create(index_name, body=request_body)

creating index clinical_trials


In [ ]:
#### INDEXING DOCUMENTS FROM PARSED DOCUMENT COLLECTION ####

In [ ]:
for title, id, brief, gender, healthy, minage, maxage, condition, intervention, inclusion, exclusion in tqdm(trials_parsed):
  doc_body = {
      'title': title,
      'id' :  id,
      'brief' : brief,
      'gender' : gender,
      'healthy' : healthy,
      'minage' : minage,
      'maxage' :  maxage,
      'condition' : condition,
      'intervention' : intervention,
      'inclusion' : inclusion,
      'exclusion' : exclusion

  }
  es.index(index_name, doc_body)

100%|██████████| 1227/1227 [00:14<00:00, 82.47it/s] 


In [ ]:
count = []
count.append(es.cat.count(index=index_name,h=['count']))
print(f'We have made and index called {index_name} with {count} documents')

We have made and index called clinical_trials with ['1168\n'] documents


In [ ]:
#### MAPPING FOR HITS ###

In [ ]:
def index_info(index_name):
  count, deleted, shards, =  es.cat.indices(index=index_name,
                                            h=['docs.count', 'docs.deleted', 'pri'])[:-1].split(' ')
  print(
      """
      #### INDEX INFO #####
      index_name = {}
      doc_count = {}
      shard_count = {}
      deleted_doc_count = {}
      """.format(index_name, count, shards, deleted)
  )


index_info(index_name)


      #### INDEX INFO #####
      index_name = clinical_trials
      doc_count = 1168
      shard_count = 1
      deleted_doc_count = 0
      


In [ ]:
#### DEFINING RETRIEVAL FUNCTION FOR USER INTERFACE ####

In [ ]:
def retriever():
    id_widget = widgets.Text(description='ID:')

    @widgets.interact
    def display_row(id=id_widget):
        row = trials[trials['id'] == id]
        if not row.empty:
            row_data = row.iloc[0]
            html = "<h2>Retrieved Document</h2>"
            html += "<ul>"
            for col, value in row_data.items():
              html += f"<li><b><span style='color: #006400;'>{col.capitalize()}:</span></b> {value}</li>"

            html += "</ul>"
            display(HTML(html))
        else:
            print(f"\n\t     No document found for ID: \n\t\t\t{id}")

In [ ]:
### RANDOM SELECTION OF QUERY + KEYBERT ###

In [ ]:
def query_maker(topics, top_n, relevant, non_relevant):
  ran_topic = random.choice(topics)
  # to select a specific topic include the index number here (e.g. Topic 1)
  #ran_topic = topics[0]
  top_pos = int(topics.index(ran_topic)) + 1

  kw_model = KeyBERT()
  ### keyword extraction
  keywords = kw_model.extract_keywords(ran_topic, top_n= top_n, stop_words= 'english')
  ### mutliple keyword extraction:
  multi_keywords = kw_model.extract_keywords(ran_topic,keyphrase_ngram_range=(1, 2), top_n= top_n, stop_words= 'english')
  ###creating a list of extracted query terms
  query = [i[0] for i in keywords]
  multi_kw_query =[i[0] for i in multi_keywords]

  age_list = [45, 48, 32, 44, 74, 55, 60, 57, 41, 22, 75, 34, 62, 70, 70, 79, 64, 78, 65,
         35, 57, 31, 39, 55, 42, 45, 53, 60, 24, 33, 37, 17, 42, 47, 15, 32, 20, 35,
         0.008, 60, 57, 19, 60, 14, 34, 30, 62, 41, 12, 0.42, 25, 34, 34, 57,
         22, 41, 41, 17, 15, 63, 45, 46, 54, 55, 25, 16, 54, 23, 67, 46, 34, 16, 0.008, 53, 55]

  # extracting age of patient from chosen query topic
  query_age = age_list[top_pos - 1]

  gender_list = ['M', 'M', 'F', 'F', 'M', 'F', 'M', 'M', 'M', 'F', 'M', 'F', 'M', 'F', 'F', 'F',
                 'F', 'M', 'M', 'F', 'M', 'F', 'M', 'M', 'F', 'F', 'M', 'M', 'M', 'F', 'F', 'M',
                 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'M', 'M', 'F', 'F', 'M', 'M', 'M', 'M', 'M',
                 'F', 'M', 'F', 'M', 'F', 'M', 'M', 'M', 'F', 'M', 'M', 'M', 'F', 'M', 'F', 'M',
                 'M', 'F', 'F', 'M', 'F', 'F', 'F', 'F', 'F', 'M', 'M']

  # extracting gender of patient from chosen query topic
  gender = gender_list[top_pos - 1]

  # extracting ids of relevant documents with attached ground truth
  top_rel = relevant[relevant['Topic_no'] == str(top_pos)]
  rel_doc = [i for i in top_rel["NCT_no"].astype('str')]

  # extracting ids of irrelevant documents with attached ground truth
  non_rel = non_relevant[non_relevant['Topic_no'] == str(top_pos)]
  nonrel_doc = [i for i in non_rel["NCT_no"].astype('str')]

  return keywords, query, multi_kw_query, top_pos, query_age, gender, rel_doc, nonrel_doc

In [ ]:
#### PRODUCING QUERY DETAILS FOR RANDOM QUERY ####

In [ ]:
bert, query, multi_kw_query, top_pos, age, gender, reldoc, nonrel_doc = query_maker(topics, 10, relevant, non_relevant)
print("Query terms: " +str(query))
print("Multiple query terms: " + str(multi_kw_query))
print("Topic number: " + str(top_pos))
print("Patient age: " + str(age))
print("Patient gender: " + str(gender))
print("Number of relevant documents: " + str(len(reldoc)))
print("Number of non-relevant documents: " + str(len(nonrel_doc)))

Query terms: ['schistocytes', 'fever', 'jaundice', 'vomiting', 'coli', 'toxin', 'diarrhea', 'disease', 'hemoglobin', 'bilirubin']
Multiple query terms: ['diarrhea abdominal', 'prior diarrhea', 'bloody diarrhea', 'cells shiga', 'schistocytes', 'fever', 'diarrhea onset', 'jaundice', 'vomiting', 'vomiting non']
Topic number: 32
Patient age: 17
Patient gender: M
Number of relevant documents: 26
Number of non-relevant documents: 449


In [ ]:
#### CREATING FUNCTION THAT RETURNS OPPOSITE GENDER OF PATIENT ####
#### This is used in the query structure to ensure trials where only the opposite gender are eligible are excluded from hits####

In [ ]:
def gender_opp(gender):
  opposite = ''
  if gender == "F":
    opposite = 'male'
  else:
      opposite = 'female'
  return opposite


In [ ]:

  oppo_gen = gender_opp(gender)
  oppo_gen

'female'

# SEARCH ENGINE IMPLEMENTATION

# DEFINING QUERY

Default = random query produced by query maker function.

To overwrite, define as a list of string terms e.g. query = ['I', 'want', 'to', 'search', 'something']

In [ ]:
query = query
print(query)

['schistocytes', 'fever', 'jaundice', 'vomiting', 'coli', 'toxin', 'diarrhea', 'disease', 'hemoglobin', 'bilirubin']


# DEFINING QUERY STRUCTURE

# Query Structure:

Fields can be any from index mapping:


      'title'
      'id'
      'brief'
      'gender'
      'healthy'
      'minage'
      'maxage'
      'condition'
      'intervention'
      'inclusion'
      'exclusion'

To boost: include ^[boost weight]
- e.g. 'title^10'

To change number of hits used for evaluation:
- adjust "size" parameter

To change minimum score needed for a hit:
- adjust "min_score" parameter

Query body includes
- Boolean "must" include for age range (minimum age less than or equal to patient age, maximum age greater than or equal to patient age)
- Boolean "must" include for multi match fields

In [ ]:
query_body = {
  "size": 10,
  "min_score": 10,
  "query": {
    "bool": {
      "must": [
        {
          "range": {
            "minage": {
              "lte": age
            }
          }
        },
        {
          "range": {
            "maxage": {
              "gte": age
            }
          }
        },
        {
          "multi_match": {
            "query": " ".join(query),
            "fields": ["title", "inclusion", "intervention"]
          }
        }
      ],
      "must_not": [
        {
          "match": {
            "exclusion": " ".join(query)
          }
        },
        {
          "match": {
            "gender": oppo_gen
          }
        }
      ]
    }
  }
}

print('### RESULTS ####')
results = es.search(index=index_name, body=query_body, explain=True)
print('Number of hits: {}'.format(results['hits']['total']['value']))
print('')

real_title = []
real_id = []
real_score = []
hit_no = int(results['hits']['total']['value'])

# metrics produced for evaluation framework
for hit in results['hits']['hits']:
  real_title.append(hit['_source']['title'])
  real_id.append(hit['_source']['id'])
  real_score.append(hit['_source'])

  # prints title, score and id for each hit
  title = hit['_source']['title']
  id = hit['_source']['id']
  score = hit['_score']
  print(f'title: "{title}", score: {score},\n id: {id}')



### RESULTS ####
Number of hits: 1

title: "Intravenous Milk Thistle (Silibinin-Legalon) for Hepatic Failure Induced by Amatoxin/Amanita Mushroom Poisoning", score: 11.767857,
 id: NCT00915681


# RETRIEVER:

To examine a document from the produced hits more closely, input the ID number (NCTXXXXXXXX) into the user interface below

In [ ]:
retriever()

interactive(children=(Text(value='', description='ID:'), Output()), _dom_classes=('widget-interact',))

In [ ]:
#### EVALUATION FRAMEWORK ####

In [ ]:
def eval(real_id, reldoc, nonrel_doc):
  # counts documents if the ID is present both in the ground truth with value 2, and in the hits (from specified "size" parameter)
  matching_ids = [item for item in real_id if item in reldoc]
  # counts documents if the ID is present in both the ground truth with value 0, and in the hits (from specified "size" parameter)
  fake_match = [item for item in real_id if item in non_relevant]

  #true positives: documents from hits are present in ground truth, with value of 2
  true_p = len(matching_ids)
  #false positives: documents from hits are present in ground truth, with value of 1 or 0
  false_p = len(real_id) - true_p
  # false negatives: documents from all relevant documents (those with ground truth 2) - documents with ground truth 2 which are retrieved
  false_n = len(reldoc) - len(matching_ids)
  # true negatives: all remaining documents not retrieved which have ground truth 1 or 0
  true_n = len(trials) - true_p - false_p - false_n

  #return true_p, false_p, true_n, false_n
  # accuracy
  es_acc = (true_p + true_n) / (true_p + false_p + true_n + false_n)
  # precision
  es_prec = true_p / (true_p + false_p)
  #recall
  es_rel = true_p / (true_p + false_n)

  return es_acc, es_prec, es_rel, true_p, false_p, true_n, false_n

In [ ]:
acc, prec, rel, true_p, false_p, true_n, false_n = eval(real_id, reldoc, nonrel_doc)
print(f"Accuracy is {acc * 100}")
print(f"Precision is {prec * 100}")
print(f"Recall is {rel * 100}")
print(f"True Positive is  is {true_p}")
print(f"False Positive is {false_p}")
print(f"True Negative is {true_n}")
print(f"False Negative is {false_n}")

Accuracy is 97.79951100244499
Precision is 0.0
Recall is 0.0
True Positive is  is 0
False Positive is 1
True Negative is 1200
False Negative is 26


In [ ]:
#### COMPARISON WITHOUT KEYBERT MODEL ####

In [ ]:
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
#### EXAMPLE OF QUERY PREPARED FOR SEARCH USING WHOLE QUERY ####

In [ ]:
topic_1 = topics[0]

In [ ]:
tokens = word_tokenize(topic_1)

# Define a set of stopwords
stop_words = set(stopwords.words('english'))

# Remove stopwords from the token list
filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
print(filtered_tokens)

['Patient', '45-year-old', 'man', 'history', 'anaplastic', 'astrocytoma', 'spine', 'complicated', 'severe', 'lower', 'extremity', 'weakness', 'urinary', 'retention', 's/p', 'Foley', 'catheter', ',', 'high-dose', 'steroids', ',', 'hypertension', ',', 'chronic', 'pain', '.', 'tumor', 'located', 'T-L', 'spine', ',', 'unresectable', 'anaplastic', 'astrocytoma', 's/p', 'radiation', '.', 'Complicated', 'progressive', 'lower', 'extremity', 'weakness', 'urinary', 'retention', '.', 'Patient', 'initially', 'presented', 'RLE', 'weakness', 'right', 'knee', 'gave', 'difficulty', 'walking', 'right', 'anterior', 'thigh', 'numbness', '.', 'MRI', 'showed', 'spinal', 'cord', 'conus', 'mass', 'biopsied', 'found', 'anaplastic', 'astrocytoma', '.', 'Therapy', 'included', 'field', 'radiation', 't10-l1', 'followed', '11', 'cycles', 'temozolomide', '7', 'days', '7', 'days', '.', 'followed', 'CPT-11', 'Weekly', 'x4', 'Avastin', 'Q2', 'weeks/', '2', 'weeks', 'rest', 'repeat', 'cycle', '.']


In [ ]:
lemmatizer = WordNetLemmatizer()
# lemmatizing all tokens in the query
lemmatized_tokens = []
for token in filtered_tokens:
   lemmatized_token = lemmatizer.lemmatize(token)
   lemmatized_tokens.append(lemmatized_token )

In [ ]:
#### EXAMPLE OF QUERY BODY WITH ENTIRE TOPIC PARSED AS QUERY ####

In [ ]:
query_body = {
  "size": 10,
  "query": {
    "bool": {
      "must": [
        {
          "range": {
            "minage": {
              "lte": age
            }
          }
        },
        {
          "range": {
            "maxage": {
              "gte": age
            }
          }
        },
        {
          "multi_match": {
            "query": " ".join(lemmatized_tokens),
            "fields": ["title", "inclusion", "intervention"]
          }
        }
      ],
      "must_not": [
        {
          "match": {
            "exclusion": " ".join(lemmatized_tokens)
          }
        },
        {
          "match": {
            "gender": oppo_gen
          }
        }
      ]
    }
  }
}

print('### RESULTS ####')
results = es.search(index=index_name, body=query_body, explain=True)
print('Number of hits: {}'.format(results['hits']['total']['value']))
print('')

real_title = []
real_id = []
real_score = []
hit_no = int(results['hits']['total']['value'])

for hit in results['hits']['hits']:
  real_title.append(hit['_source']['title'])
  real_id.append(hit['_source']['id'])
  real_score.append(hit['_source'])

  title = hit['_source']['title']
  id = hit['_source']['id']
  score = hit['_score']
  print(f'title: "{title}", score: {score}, id: {id}')



### RESULTS ####
Number of hits: 56

title: "Intraocular Position of Secondary Iris-Fixated IOLs in Aphakic Eyes", score: 49.942543, id: NCT00773266
title: "Temodal (Temozolomide) Post Marketing Surveillance Protocol (Study P05557AM2)", score: 42.10403, id: NCT00723827
title: "Cixutumumab in Treating Patients With Relapsed or Refractory Solid Tumors", score: 19.572813, id: NCT00831844
title: "Analysis of Exhaled Breath by Biosensors in Adults With Asthma", score: 18.464317, id: NCT00819676
title: "Carboplatin and Docetaxel Followed by Epstein-Barr Virus Cytotoxic T Lymphocytes", score: 17.313124, id: NCT00953420
title: "Haploidentical Hematopoietic Stem Cell Transplantation Using A Novel Clofarabine Containing Conditioning Regimen For Patients With Refractory Hematologic Malignancies", score: 16.468113, id: NCT00824135
title: "Characterization of WAGR Syndrome and Other Chromosome 11 Gene Deletions", score: 15.479066, id: NCT00758108
title: "Intravenous Milk Thistle (Silibinin-Legalon)

In [ ]:
acc, prec, rel, true_p, false_p, true_n, false_n = eval(real_id, reldoc, nonrel_doc)
print(f"Accuracy is {acc * 100}")
print(f"Precision is {prec * 100}")
print(f"Relevance is {rel * 100}")
print(f"True Positive is  is {true_p}")
print(f"False Positive is {false_p}")
print(f"True Negative is {true_n}")
print(f"False Negative is {false_n}")

Accuracy is 97.06601466992664
Precision is 0.0
Relevance is 0.0
True Positive is  is 0
False Positive is 10
True Negative is 1191
False Negative is 26
